# ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Importing Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
   

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Get total number of rows 
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setting up Apache Cassandra

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Creating Keyspace

In [ ]:
session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Setting Keyspace

In [ ]:
session.set_keyspace('sparkify')

## Query 1: Returns the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
##Description: Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. 
##Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

session.execute("""
        CREATE TABLE IF NOT EXISTS sessionId_338_itemInSession_4_song (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)                  

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO sessionId_338_itemInSession_4_song (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Verifying that the data have been inserted into each table

In [ ]:
verify_query_1 = "SELECT artist, song, length FROM sessionId_338_itemInSession_4_song WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(verify_query_1)
for row in rows:
    print (row.artist, row.song, row.length)

## Query 2: Returns name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

In [ ]:
##Query 2: returns only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

session.execute("""
        CREATE TABLE IF NOT EXISTS sessionid_182_userid_10_songs (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessionid_182_userid_10_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))
        
    
verify_query_2 = "SELECT artist, song, firstName, lastName FROM sessionid_182_userid_10_songs WHERE sessionId = 182 and userId = 10  ORDER BY itemInSession"
rows = session.execute(verify_query_2)
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)                    

## Query 3: Returns every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
##Query 3: returns every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""
        CREATE TABLE IF NOT EXISTS first_last_name_song (
            userId INT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY (song, userId));""")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO first_last_name_song (userId, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), str(line[9]), str(line[1]), str(line[4])))
        
    
verify_query_3 = "SELECT firstName, lastName FROM first_last_name_song WHERE song = 'All Hands Against His Own'"
rows = session.execute(verify_query_3)
for row in rows:
    print (row.firstname, row.lastname)

### Dropping the tables before closing out the sessions

In [ ]:
session.execute("drop table sessionId_338_itemInSession_4_song")
session.execute("drop table sessionid_182_userid_10_songs")
session.execute("drop table first_last_name_song")

### Closing the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()